# Creating Recommenders and Solutions <a class="anchor" id="top"></a>

## Outline

1. [Introduction](#intro)
1. [Create Domain Recommenders](#recommenders)
1. [Create Solutions](#solutions)
1. [Evaluate Solutions](#eval)
1. [Using Evaluation Metrics](#use)
1. [Deploy a Campaign](#deploy)
1. [Create Filters](#interact)
1. [Storing useful variables](#wrapup)

## Introduction <a class="anchor" id="intro"></a>

To recap, we created a Domain dataset group, where you create preconfigured resources for different business domains and use cases, such as getting recommendations for similar videos (VIDEO_ON_DEMAND domain) or best selling items (ECOMMERCE domain). You choose your business domain, import your data, and create recommenders. You use recommenders in your application to get recommendations.

Use a [Domain dataset group](https://docs.aws.amazon.com/personalize/latest/dg/domain-dataset-groups.html) if you have a video on demand or e-commerce application and want Amazon Personalize to find the best configurations for your use cases. If you start with a Domain dataset group, you can also add custom resources such as solutions with solution versions trained with recipes for custom use cases.

A [Custom dataset group](https://docs.aws.amazon.com/personalize/latest/dg/custom-dataset-groups.html), where you create configurable resources for custom use cases and batch recommendation workflows. You choose a recipe, train a solution version (model), and deploy the solution version with a campaign. You use a campaign in your application to get recommendations.

Use a Custom dataset group if you don't have a video on demand or e-commerce application or want to configure and manage only custom resources, or want to get recommendations in a batch workflow. If you start with a Custom dataset group, you can't associate it with a domain later. Instead, create a new Domain dataset group.

Regardless of the use case, the algorithms all share a base of learning on user-item-interaction data which is defined by 3 core attributes:

1. **UserID** - The user who interacted
1. **ItemID** - The item the user interacted with
1. **Timestamp** - The time at which the interaction occurred

We also support optional event types and event values defined by:

1. **Event Type** - Categorical label of an event (clicked, purchased, rated, listened, watched, etc).
1. **Event Value** - A numeric value corresponding to the event type that occurred. This value can be used to filter interactions that are included in model training by specifying a minimum threshold. For example, suppose you have an event type of `Rated` and the value is the user rating on a scale of 0 to 5. Since Personalize models on positive interactions, you can use an event value threshold of, say, 3 to only include interactions with an event type of `Rated` that have an event value of 3 or higher.

The event type and event value fields are additional data which can be used to filter the data sent for training the personalization model. In this particular exercise we dol not have an event value and will not be filtering the training data, i.e. we will use all available data for training (More information on how to use the eventValue with eventValueThreshold for Custom Domains in the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/recording-events.html)). 

### In this notebook we will accomplish the following:

Create Video on Demand Domain Recommenders for the following use cases:

1. [More like X](https://docs.aws.amazon.com/personalize/latest/dg/VIDEO_ON_DEMAND-use-cases.html#more-like-y-use-case): recommendations for movies that are similar to a movie that you specify. With this use case, Amazon Personalize automatically filters movies the user watched based on the userId that you specify and Watch events.

1. [Top picks for you](https://docs.aws.amazon.com/personalize/latest/dg/VIDEO_ON_DEMAND-use-cases.html#top-picks-use-case): personalized content recommendations for a user that you specify. With this use case, Amazon Personalize automatically filters videos the user watched based on the userId that you specify and Watch events.

Create a custom solution and solution versions for the following use case:

3. [Personalized-Ranking](https://docs.aws.amazon.com/personalize/latest/dg/working-with-predefined-recipes.html): will be used to rerank a list of movies.

![Workflow](images/image2.png)

To run this notebook, you need to have run the previous notebook, `01_Data_Layer.ipynb`, where you created a dataset and imported interaction, item, and user metadata data into Amazon Personalize. At the end of that notebook, you saved some of the variable values, which you now need to load into this notebook.

In [8]:
%store -r

Similar to the previous notebook, start by importing the relevant packages, and set up a connection to Amazon Personalize using the SDK.

In [9]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random
import boto3
import botocore
from botocore.exceptions import ClientError
import pandas as pd

In [10]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## Create Domain Recommenders <a class="anchor" id="recommenders"></a>
[Back to top](#top)



We'll start with pre-configured VIDEO_ON_DEMAND Recommenders that match some of our core use cases. Each domain has different use cases. When you create a recommender you create it for a specific use case, and each use case has different requirements for getting recommendations.

Let us look at the Recommenders supported for this domain:

In [11]:
available_recipes = personalize.list_recipes(domain='VIDEO_ON_DEMAND')
display_available_recipes = available_recipes ['recipes']
available_recipes = personalize.list_recipes(domain='VIDEO_ON_DEMAND',nextToken=available_recipes['nextToken'])#paging to get the rest of the recipes 
display_available_recipes = display_available_recipes + available_recipes['recipes']
display(display_available_recipes)

[{'name': 'aws-vod-because-you-watched-x',
  'recipeArn': 'arn:aws:personalize:::recipe/aws-vod-because-you-watched-x',
  'status': 'ACTIVE',
  'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
  'lastUpdatedDateTime': datetime.datetime(2022, 7, 14, 15, 4, 27, 618000, tzinfo=tzlocal()),
  'domain': 'VIDEO_ON_DEMAND'},
 {'name': 'aws-vod-more-like-x',
  'recipeArn': 'arn:aws:personalize:::recipe/aws-vod-more-like-x',
  'status': 'ACTIVE',
  'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
  'lastUpdatedDateTime': datetime.datetime(2022, 7, 14, 15, 4, 27, 617000, tzinfo=tzlocal()),
  'domain': 'VIDEO_ON_DEMAND'},
 {'name': 'aws-vod-most-popular',
  'recipeArn': 'arn:aws:personalize:::recipe/aws-vod-most-popular',
  'status': 'ACTIVE',
  'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
  'lastUpdatedDateTime': datetime.datetime(2022, 7, 14, 15, 4, 27, 617000, tzinfo=tzlocal()),
  'domain': 'VIDEO_ON_DEMAND'},


[More use cases per domain](https://docs.aws.amazon.com/personalize/latest/dg/domain-use-cases.html).

### Create a "More like X" Recommender

We are going to create a recommender of the type "More like X". This type of recommender offers recommendations for videos that are similar to a video a user watched. With this use case, Amazon Personalize automatically filters videos the user watched based on the userId specified in the `get_recommendations` call. 

In [12]:
create_recommender_response = personalize.create_recommender(
  name = 'more-like-x',
  recipeArn = 'arn:aws:personalize:::recipe/aws-vod-more-like-x',
  datasetGroupArn = dataset_group_arn
)
recommender_more_like_x_arn = create_recommender_response["recommenderArn"]
print (json.dumps(create_recommender_response))

{"recommenderArn": "arn:aws:personalize:us-east-1:061703360474:recommender/more-like-x", "ResponseMetadata": {"RequestId": "0ce3675d-9d43-49df-996a-2ea189534e65", "HTTPStatusCode": 200, "HTTPHeaders": {"date": "Tue, 16 Aug 2022 14:26:23 GMT", "content-type": "application/x-amz-json-1.1", "content-length": "87", "connection": "keep-alive", "x-amzn-requestid": "0ce3675d-9d43-49df-996a-2ea189534e65"}, "RetryAttempts": 0}}


### Create a "Top picks for you" Recommender

We are going to create a second recommender of the type "Top picks for you". This type of recommender offers personalized streaming content recommendations for a user that you specify. With this use case, Amazon Personalize automatically filters videos the user watched based on the userId that you specify and Watch events.

In [13]:
create_recommender_response = personalize.create_recommender(
  name = 'top_picks_for_you',
  recipeArn = 'arn:aws:personalize:::recipe/aws-vod-top-picks',
  datasetGroupArn = dataset_group_arn
)
recommender_top_picks_arn = create_recommender_response["recommenderArn"]
print (json.dumps(create_recommender_response))

{"recommenderArn": "arn:aws:personalize:us-east-1:061703360474:recommender/top_picks_for_you", "ResponseMetadata": {"RequestId": "f9785e16-4209-46ef-aa5a-67895e07b0f6", "HTTPStatusCode": 200, "HTTPHeaders": {"date": "Tue, 16 Aug 2022 14:26:27 GMT", "content-type": "application/x-amz-json-1.1", "content-length": "93", "connection": "keep-alive", "x-amzn-requestid": "f9785e16-4209-46ef-aa5a-67895e07b0f6"}, "RetryAttempts": 0}}


## Create Solutions <a class="anchor" id="solutions"></a>
[Back to top](#top)

Some use cases require a custom implementation. 

In Amazon Personalize, a specific variation of an algorithm is called a recipe. Different recipes are suitable for different situations. A trained model is called a solution, and each solution can have many versions that relate to a given volume of data when the model was trained.

Let's look at all available recipes that are not of a specific domain and can be used to create custom solutions. 

In [14]:
available_recipes = personalize.list_recipes()
display_available_recipes = available_recipes ['recipes']
available_recipes = personalize.list_recipes(nextToken=available_recipes['nextToken'])#paging to get the rest of the recipes 
display_available_recipes = display_available_recipes + available_recipes['recipes']

display ([recipe  for recipe in display_available_recipes if 'domain' not in recipe])


[{'name': 'aws-item-affinity',
  'recipeArn': 'arn:aws:personalize:::recipe/aws-item-affinity',
  'status': 'ACTIVE',
  'creationDateTime': datetime.datetime(2021, 7, 15, 0, 0, tzinfo=tzlocal()),
  'lastUpdatedDateTime': datetime.datetime(2022, 7, 14, 15, 4, 27, 617000, tzinfo=tzlocal())},
 {'name': 'aws-item-attribute-affinity',
  'recipeArn': 'arn:aws:personalize:::recipe/aws-item-attribute-affinity',
  'status': 'ACTIVE',
  'creationDateTime': datetime.datetime(2021, 8, 25, 0, 0, tzinfo=tzlocal()),
  'lastUpdatedDateTime': datetime.datetime(2022, 7, 14, 15, 4, 27, 617000, tzinfo=tzlocal())},
 {'name': 'aws-personalized-ranking',
  'recipeArn': 'arn:aws:personalize:::recipe/aws-personalized-ranking',
  'status': 'ACTIVE',
  'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
  'lastUpdatedDateTime': datetime.datetime(2022, 7, 14, 15, 4, 27, 617000, tzinfo=tzlocal())},
 {'name': 'aws-popularity-count',
  'recipeArn': 'arn:aws:personalize:::recipe/aws-popularity

We want to rank a list of items for a specific user. This is useful if you have a collection of ordered items, such as search results, promotions, or curated lists, and you want to provide a personalized re-ranking for each of your users. To implement this use case, we will create a custom solution using the recipe.

The [Personalized-Ranking](https://docs.aws.amazon.com/personalize/latest/dg/working-with-predefined-recipes.html) recipe provides recommendations in ranked order based on predicted interest level. This recipe generates personalized rankings of items. A personalized ranking is a list of recommended items that are re-ranked for a specific user. This is useful if you have a collection of ordered items, such as search results, promotions, or curated lists, and you want to provide a personalized re-ranking for each of your users.

These custom solution will use the same datasets that we already implemented so all we need to do is create a solution and solution version for this recipe.

### Personalized Ranking

Personalized Ranking is an interesting application of HRNN. Instead of just recommending what is most probable for the user in question, this algorithm takes in a list of items as well as a user. The items are then returned back in the order of most probable relevance for the user. The use case here is for filtering on unique categories that you do not have item metadata to create a filter, or when you have a broad collection that you would like better ordered for a particular user.

For our use case, using the MovieLens data, we could imagine that a Video on Demand application may want to create a shelf of comic book movies, or movies by a specific director. We can generate these lists based on metadata we have. We would use personalized ranking to re-order the list of movies for each user. 

We start by selecting the recipe.

In [15]:
rerank_recipe_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking"

### Create the solution

First you create a solution using the recipe. Although you provide the dataset ARN in this step, the model is not yet trained. See this as an identifier instead of a trained model.

In [16]:
rerank_create_solution_response = personalize.create_solution(
    name = "personalize-poc-rerank",
    datasetGroupArn = dataset_group_arn,
    recipeArn = rerank_recipe_arn
)

rerank_solution_arn = rerank_create_solution_response['solutionArn']
print(json.dumps(rerank_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:061703360474:solution/personalize-poc-rerank",
  "ResponseMetadata": {
    "RequestId": "49fcb973-77f4-47f8-b302-bbf15c7c1564",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 16 Aug 2022 14:27:07 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "92",
      "connection": "keep-alive",
      "x-amzn-requestid": "49fcb973-77f4-47f8-b302-bbf15c7c1564"
    },
    "RetryAttempts": 0
  }
}


### Create the solution version

Once you have a solution, you need to create a version in order to complete the model training. The training can take a while to complete, upwards of 25 minutes, and an average of 35 minutes for this recipe with our dataset. Normally, we would use a while loop to poll until the task is completed. 

In [17]:
rerank_create_solution_version_response = personalize.create_solution_version(
    solutionArn = rerank_solution_arn
)

In [18]:
rerank_solution_version_arn = rerank_create_solution_version_response['solutionVersionArn']
print(json.dumps(rerank_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:061703360474:solution/personalize-poc-rerank/d99ff5fd",
  "ResponseMetadata": {
    "RequestId": "ba15ffcc-5e11-4413-a766-2b5b9d465fd5",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 16 Aug 2022 14:27:11 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "108",
      "connection": "keep-alive",
      "x-amzn-requestid": "ba15ffcc-5e11-4413-a766-2b5b9d465fd5"
    },
    "RetryAttempts": 0
  }
}


### User Segmentation

Amazon Personalize offers two recipes that segment your users based on their interest in different product categories, brands and more.

1. Our item affinity recipe aws-item-affinity identifies users based on their interest in the individual items in your catalog such as a movie, song, or product.
1. The item attribute affinity recipe aws-item-attribute identifies users based on the attributes of items in your catalog such as genre or brand. This allows you to better engage users with your marketing campaigns and improve retention through targeted messaging.

The cells below demonstrate how to use the aws-item-affinity and aws-item-attribute recipe to create user segments based on their preferences for the movie lens dataset. We use one dataset group which contains user-item interaction data and item metadata. We use these datasets to train solutions using the two recipes and create user segments.

We start by selecting the recipe.

In [44]:
item_user_recipe_arn = "arn:aws:personalize:::recipe/aws-item-affinity"
item_attribute_user_recipe_arn = "arn:aws:personalize:::recipe/aws-item-attribute-affinity"

### Create the solution

First you create a solution using the recipe. Although you provide the dataset ARN in this step, the model is not yet trained. See this as an identifier instead of a trained model.

In [45]:
itemaffinity_create_solution_response = personalize.create_solution(
    name = "item-affinity-movie-lens",
    datasetGroupArn = dataset_group_arn,
    recipeArn = item_user_recipe_arn
)

itemaffinity_solution_arn = itemaffinity_create_solution_response['solutionArn']
print(json.dumps(itemaffinity_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:061703360474:solution/item-affinity-movie-lens",
  "ResponseMetadata": {
    "RequestId": "cf792323-5f31-452f-9578-0ef505938b1f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Mon, 29 Aug 2022 15:51:01 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "94",
      "connection": "keep-alive",
      "x-amzn-requestid": "cf792323-5f31-452f-9578-0ef505938b1f"
    },
    "RetryAttempts": 0
  }
}


In [46]:
itemaffinity_attribute_create_solution_response = personalize.create_solution(
    name = "item-affinity-attribute-movie-lens",
    datasetGroupArn = dataset_group_arn,
    recipeArn = item_attribute_user_recipe_arn
)

itemaffinity_attribute_solution_arn = itemaffinity_attribute_create_solution_response['solutionArn']
print(json.dumps(itemaffinity_attribute_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:061703360474:solution/item-affinity-attribute-movie-lens",
  "ResponseMetadata": {
    "RequestId": "c399705e-f9a5-4eda-b555-ea98656b8d6b",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Mon, 29 Aug 2022 15:52:09 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "104",
      "connection": "keep-alive",
      "x-amzn-requestid": "c399705e-f9a5-4eda-b555-ea98656b8d6b"
    },
    "RetryAttempts": 0
  }
}


### Create the solution version

Once you have a solution, you need to create a version in order to complete the model training. The training can take a while to complete.  Normally, we would use a while loop to poll until the task is completed. 

In [48]:
itemaffinity_create_solution_version_response = personalize.create_solution_version(
    solutionArn = itemaffinity_solution_arn
)

In [49]:
itemaffinity_solution_version_arn = itemaffinity_create_solution_version_response['solutionVersionArn']
print(json.dumps(itemaffinity_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:061703360474:solution/item-affinity-movie-lens/cd4d31a9",
  "ResponseMetadata": {
    "RequestId": "2c3044ae-e155-49de-8f79-3604cbf67d78",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Mon, 29 Aug 2022 15:53:42 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "110",
      "connection": "keep-alive",
      "x-amzn-requestid": "2c3044ae-e155-49de-8f79-3604cbf67d78"
    },
    "RetryAttempts": 0
  }
}


In [51]:
itemaffinity_attribute_create_solution_version_response = personalize.create_solution_version(
    solutionArn = itemaffinity_attribute_solution_arn
)

In [52]:
itemaffinity_attribute_solution_version_arn = itemaffinity_attribute_create_solution_version_response['solutionVersionArn']
print(json.dumps(itemaffinity_attribute_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:061703360474:solution/item-affinity-attribute-movie-lens/f291d268",
  "ResponseMetadata": {
    "RequestId": "592b943a-662c-483c-a27f-346346ad3abc",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Mon, 29 Aug 2022 15:55:22 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "120",
      "connection": "keep-alive",
      "x-amzn-requestid": "592b943a-662c-483c-a27f-346346ad3abc"
    },
    "RetryAttempts": 0
  }
}


### View solution and Recommender creation status

To view the status updates in the console:

* In another browser tab you should already have the AWS Console up from opening this notebook instance. 
* Switch to that tab and search at the top for the service `Personalize`, then go to that service page. 
* Click `Dataset groups`.
* Click the name of your dataset group, if you did not change it, it is "personalize-poc-movielens".
* Click `Recommenders`.
* You will see a list of the two recommenders you created above, including a column with the status of the recommender. Once it is `Active`, your recommender is ready.
* Click on `Custom Resources`. This oppens up the list of custom resources that youhave created.
* Click on `Solutions and Recipes` to see your re-ranking solutions. If you click on `personalize-poc-rerank` you can see the status of the solution versions. Once it is `Active`, your solution is ready to be reviewed. It is also capable of being deployed.

Or simply run the cell below to keep track of the recommenders and solution version creation status.

In [ ]:
%%time

max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:

    # Recommender more_like_x
    version_response = personalize.describe_recommender(
        recommenderArn = recommender_more_like_x_arn
    )
    status_more_like_x = version_response["recommender"]["status"]

    if status_more_like_x == "ACTIVE":
        print("Build succeeded for {}".format(recommender_more_like_x_arn))
        
    elif status_more_like_x == "CREATE FAILED":
        print("Build failed for {}".format(recommender_more_like_x_arn))
        break

    if not status_more_like_x == "ACTIVE":
        print("The recommender more_like_x build is still in progress")
    else:
        print("The recommender more_like_x is ACTIVE")

    # Recommender top_picks_for_you
    version_response = personalize.describe_recommender(
        recommenderArn = recommender_top_picks_arn
    )
    status_top_picks = version_response["recommender"]["status"]

    if status_top_picks == "ACTIVE":
        print("Build succeeded for {}".format(recommender_top_picks_arn))
    elif status_top_picks == "CREATE FAILED":
        print("Build failed for {}".format(recommender_top_picks_arn))
        break

    if not status_top_picks == "ACTIVE":
        print("The recommender top_picks build is still in progress")
    else:
        print("The recommender top_picks is ACTIVE")
        
    # Reranking Solution 
    version_response = personalize.describe_solution_version(
        solutionVersionArn = rerank_solution_version_arn
    )
    status_rerank_solution = version_response["solutionVersion"]["status"]

    if status_rerank_solution == "ACTIVE":
        print("Build succeeded for {}".format(rerank_solution_version_arn))
        
    elif status_rerank_solution == "CREATE FAILED":
        print("Build failed for {}".format(rerank_solution_version_arn))
        break

    if not status_rerank_solution == "ACTIVE":
        print("Rerank Solution Version build is still in progress")
    else:
        print("The Rerank solution is ACTIVE")
        
    # Item affinity Solution 
    version_response = personalize.describe_solution_version(
        solutionVersionArn = itemaffinity_solution_version_arn
    )
    status_itemaffinity_solution = version_response["solutionVersion"]["status"]

    if status_itemaffinity_solution == "ACTIVE":
        print("Build succeeded for {}".format(itemaffinity_solution_version_arn))
        
    elif status_itemaffinity_solution == "CREATE FAILED":
        print("Build failed for {}".format(itemaffinity_solution_version_arn))
        break

    if not status_itemaffinity_solution == "ACTIVE":
        print("Item Affinity Version build is still in progress")
    else:
        print("The Item Affinity solution is ACTIVE")
        
    # Item affinity attribute Solution
    version_response = personalize.describe_solution_version(
        solutionVersionArn = itemaffinity_attribute_solution_version_arn
    )
    status_itemaffinity_attribute_solution = version_response["solutionVersion"]["status"]

    if status_itemaffinity_attribute_solution == "ACTIVE":
        print("Build succeeded for {}".format(itemaffinity_attribute_solution_version_arn))
        
    elif status_itemaffinity_attribute_solution == "CREATE FAILED":
        print("Build failed for {}".format(itemaffinity_attribute_solution_version_arn))
        break

    if not status_itemaffinity_attribute_solution == "ACTIVE":
        print("Rerank Solution Version build is still in progress")
    else:
        print("The Rerank solution is ACTIVE")
                
        
    # check status on all     
        
    if status_more_like_x == "ACTIVE" and status_top_picks == 'ACTIVE' and status_rerank_solution == "ACTIVE" and status_itemaffinity_solution == "ACTIVE" and status_itemaffinity_attribute_solution == "ACTIVE":
        break

    print()
    time.sleep(60)

Build succeeded for arn:aws:personalize:us-east-1:061703360474:recommender/more-like-x
The recommender more_like_x is ACTIVE
Build succeeded for arn:aws:personalize:us-east-1:061703360474:recommender/top_picks_for_you
The recommender top_picks is ACTIVE
Build succeeded for arn:aws:personalize:us-east-1:061703360474:solution/personalize-poc-rerank/d99ff5fd
The Rerank solution is ACTIVE
Item Affinity Version build is still in progress
Rerank Solution Version build is still in progress

Build succeeded for arn:aws:personalize:us-east-1:061703360474:recommender/more-like-x
The recommender more_like_x is ACTIVE
Build succeeded for arn:aws:personalize:us-east-1:061703360474:recommender/top_picks_for_you
The recommender top_picks is ACTIVE
Build succeeded for arn:aws:personalize:us-east-1:061703360474:solution/personalize-poc-rerank/d99ff5fd
The Rerank solution is ACTIVE
Item Affinity Version build is still in progress
Rerank Solution Version build is still in progress

Build succeeded for ar

### Wait for Recommenders and Solution Versions to finish training

It can take 40-60 minutes for all Recommenders and solution versions to be created. During this process a model is being trained and tested with the data contained within your datasets. The duration of training jobs can increase based on the size of the dataset, training parameters and a selected recipe. 

While you are waiting for this process to complete you can learn more about solutions in [the documentation](https://docs.aws.amazon.com/personalize/latest/dg/training-deploying-solutions.html)

## Evaluate Solutions<a class="anchor" id="eval"></a>
[Back to top](#top)

Custom Solutions and Solution Versions give you the freedom to fine tune parameters and evaluate them.

### Hyperparameter tuning

Personalize offers the option of running hyperparameter tuning when creating a solution. Because of the additional computation required to perform hyperparameter tuning, this feature is turned off by default. Therefore, the solution we created above, will simply use the default values of the hyperparameters for the recipe. For more information about hyperparameter tuning, see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/customizing-solution-config-hpo.html).

If you have settled on the correct recipe to use, and are ready to run hyperparameter tuning, the following code shows how you would do so, using Personalized Ranking as an example.

```python
rerank_create_solution_response = personalize.create_solution(
    name = "personalize-poc-rerank-hpo",
    datasetGroupArn = dataset_group_arn,
    recipeArn = rerank_recipe_arn,
    performHPO=True
)

rerank_solution_arn = rerank_create_solution_response['solutionArn']
print(json.dumps(rerank_create_solution_response, indent=2))
```

If you already know the values you want to use for a specific hyperparameter, you can also set this value when you create the solution. The code below shows how you could set the value for the `recency_mask` for the Personalized Ranking recipe. More information on this hyperparameter in the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-search.html).

```python
rerank_create_solution_response = personalize.create_solution(
    name = "personalize-poc-rerank-set-hp",
    datasetGroupArn = dataset_group_arn,
    recipeArn = rerank_recipe_arn,
    solutionConfig = {
        'algorithmHyperParameters': {
            'recency_mask': False
        }
    }
)

rerank_solution_arn = rerank_create_solution_response['solutionArn']
print(json.dumps(rerank_create_solution_response, indent=2))
```

### Evaluate solution versions 

It should not take more than an hour to train all the recommenders and solutions from this notebook. While training is in progress, we recommend taking the time to read up on the various algorithms (recipes) and their behavior in detail. This is also a good time to consider alternatives to how the data was fed into the system and what kind of results you expect to see.

When the solution finishes creating, the next step is to obtain the evaluation metrics. Personalize calculates these metrics based on a subset of the training data. The image below illustrates how Personalize splits the data. Given 10 users, with 10 interactions each (a circle represents an interaction), the interactions are ordered from oldest to newest based on the timestamp. Personalize uses all of the interaction data from 90% of the users (blue circles) to train the solution version, and the remaining 10% for evaluation. For each of the users in the remaining 10%, 90% of their interaction data (green circles) is used as input for the call to the trained model. The remaining 10% of their data (orange circle) is compared to the output produced by the model and used to calculate the evaluation metrics.

![personalize metrics](../imgs/personalize_metrics.png)

We recommend reading [the documentation](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html) to understand the metrics, but we have also copied parts of the documentation below for convenience.

You need to understand the following terms regarding evaluation in Personalize:

* *Relevant recommendation* refers to a recommendation that matches a value in the testing data for the particular user.
* *Rank* refers to the position of a recommended item in the list of recommendations. Position 1 (the top of the list) is presumed to be the most relevant to the user.
* *Query* refers to the internal equivalent of a GetRecommendations call.

The metrics produced by Personalize are:
* **coverage**: The proportion of unique recommended items from all queries out of the total number of unique items in the training data (includes both the Items and Interactions datasets).
* **mean_reciprocal_rank_at_25**: The [mean of the reciprocal ranks](https://en.wikipedia.org/wiki/Mean_reciprocal_rank) of the first relevant recommendation out of the top 25 recommendations over all queries. This metric is appropriate if you're interested in the single highest ranked recommendation.
* **normalized_discounted_cumulative_gain_at_K**: Discounted gain assumes that recommendations lower on a list of recommendations are less relevant than higher recommendations. Therefore, each recommendation is discounted (given a lower weight) by a factor dependent on its position. To produce the [cumulative discounted gain](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) (DCG) at K, each relevant discounted recommendation in the top K recommendations is summed together. The normalized discounted cumulative gain (NDCG) is the DCG divided by the ideal DCG such that NDCG is between 0 - 1. (The ideal DCG is where the top K recommendations are sorted by relevance.) Amazon Personalize uses a weighting factor of 1/log(1 + position), where the top of the list is position 1. This metric rewards relevant items that appear near the top of the list, because the top of a list usually draws more attention.
* **precision_at_K**: The number of relevant recommendations out of the top K recommendations divided by K. This metric rewards precise recommendation of the relevant items.

Let's take a look at the evaluation metrics for each of the solutions produced in this notebook. Please note that your results might differ from the results described in the text of this notebook, due to the quality of the Movielens dataset. 

### Personalized ranking metrics

Retrieve the evaluation metrics for the personalized ranking solution version.

In [20]:
rerank_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = rerank_solution_version_arn
)

print(json.dumps(rerank_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:061703360474:solution/personalize-poc-rerank/d99ff5fd",
  "metrics": {
    "coverage": 0.1409,
    "mean_reciprocal_rank_at_25": 0.4902,
    "normalized_discounted_cumulative_gain_at_10": 0.4228,
    "normalized_discounted_cumulative_gain_at_25": 0.4901,
    "normalized_discounted_cumulative_gain_at_5": 0.3812,
    "precision_at_10": 0.1014,
    "precision_at_25": 0.0637,
    "precision_at_5": 0.1481
  },
  "ResponseMetadata": {
    "RequestId": "0591ad8b-b456-4d25-8c53-da4c207b8a2a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 16 Aug 2022 18:29:16 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "406",
      "connection": "keep-alive",
      "x-amzn-requestid": "0591ad8b-b456-4d25-8c53-da4c207b8a2a"
    },
    "RetryAttempts": 0
  }
}


In [58]:
get_solution_metrics_response = personalize.get_solution_metrics(solutionVersionArn=itemaffinity_solution_version_arn)
print(get_solution_metrics_response['metrics'])

{'coverage': 0.7817, 'hits_at_1_percent': 0.855, 'recall_at_1_percent': 0.1105}


In [59]:
get_solution_metrics_response = personalize.get_solution_metrics(solutionVersionArn=itemaffinity_attribute_solution_version_arn)
print(get_solution_metrics_response['metrics'])

{'coverage': 0.3794, 'hits_at_1_percent': 6.2353, 'recall_at_1_percent': 0.0281}


Just a quick comment on this one, here we see again a precision of near (2.7% for full, 2.2% for small), as this is based on User Personalization, that is to be expected. However the sample items are not the same items used for validaiton, thus the low scoring.

## Using Evaluation Metrics <a class="anchor" id="use"></a>
[Back to top](#top)

It is important to use evaluation metrics carefully. There are a number of factors to keep in mind.

* If there is an existing recommendation system in place, this will have influenced the user's interaction history which you use to train your new solutions. This means the evaluation metrics are biased to favor the existing solution. If you work to push the evaluation metrics to match or exceed the existing solution, you may just be pushing the User Personalization to behave like the existing solution and might not end up with something better.
* The HRNN Coldstart recipe is difficult to evaluate using the metrics produced by Amazon Personalize. The aim of the recipe is to recommend items which are new to your business. Therefore, these items will not appear in the existing user transaction data which is used to compute the evaluation metrics. As a result, HRNN Coldstart will never appear to perform better than the other recipes, when compared on the evaluation metrics alone. Note: The User Personalization recipe also includes improved cold start functionality

Keeping in mind these factors, the evaluation metrics produced by Personalize are generally useful for two cases:
1. Comparing the performance of solution versions trained on the same recipe, but with different values for the hyperparameters and features (impression data etc)
1. Comparing the performance of solution versions trained on different recipes (except HRNN Coldstart). Here also keep in mind that the recipes answer different use cases and comparing them to each other might not make sense in your solution.

Properly evaluating a recommendation system is always best done through A/B testing while measuring actual business outcomes. Since recommendations generated by a system usually influence the user behavior which it is based on, it is better to run small experiments and apply A/B testing for longer periods of time. Over time, the bias from the existing model will fade.

## Deploy a Campaign <a class="anchor" id="deploy"></a>
[Back to top](#top)

Once a solution version is created, it is possible to get recommendations from them, and to get a feel for their overall behavior.

For real-time recommendations, after you prepare and import data and creating a solution, you are ready to deploy your solution version to generate recommendations. You deploy a solution version by creating an Amazon Personalize campaign. If you are getting batch recommendations, you don't need to create a campaign. For more information see [Getting batch recommendations and user segments](https://docs.aws.amazon.com/personalize/latest/dg/recommendations-batch.html).

We will deploy a campaign for the solution version. 

### Create a campaign 

A campaign is a hosted solution version; an endpoint which you can query for recommendations. Pricing is set by estimating throughput capacity (requests from users for personalization per second). When deploying a campaign, you set a minimum throughput per second (TPS) value. This service, like many within AWS, will automatically scale based on demand, but if latency is critical, you may want to provision ahead for larger demand. For this POC and demo, all minimum throughput thresholds are set to 1. For more information, see the [pricing page](https://aws.amazon.com/personalize/pricing/).

Once we're satisfied with our solution version, we need to create Campaigns for each solution version. When creating a campaign you specify the minimum transactions per second (`minProvisionedTPS`) that you expect to make against the service for this campaign. Personalize will automatically scale the inference endpoint up and down for the campaign to match demand but will never scale below `minProvisionedTPS`.

Let's create a campaigns for our solution versions set at `minProvisionedTPS` of 1.

In [21]:
rerank_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-rerank",
    solutionVersionArn = rerank_solution_version_arn,
    minProvisionedTPS = 1
)

rerank_campaign_arn = rerank_create_campaign_response['campaignArn']
print(json.dumps(rerank_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:061703360474:campaign/personalize-poc-rerank",
  "ResponseMetadata": {
    "RequestId": "4495fbe9-6a0c-407a-a652-a4f64193fa1d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 16 Aug 2022 18:29:23 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "92",
      "connection": "keep-alive",
      "x-amzn-requestid": "4495fbe9-6a0c-407a-a652-a4f64193fa1d"
    },
    "RetryAttempts": 0
  }
}


### View campaign creation status

This is how you view the status updates in the console:

* In another browser tab you should already have the AWS Console open from opening this notebook instance. 
* Switch to that tab and search at the top for the service `Personalize`, then go to that service page. 
* Click `Dataset groups`.
* Click the name of your dataset group.
* Click `Recommenders`
* Click `Custom Resources`
* Click `Campaigns`.
* You will now see a list of all of the campaigns you created above, including a column with the status of the campaign. Once it is `Active`, your campaign is ready to be queried.

Or simply run the cell below to keep track of the campaign creation status of the campaign we created.

While you are waiting for this to complete you can learn more about campaigns in [the documentation](https://docs.aws.amazon.com/personalize/latest/dg/campaigns.html)

In [22]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:

    version_response = personalize.describe_campaign(
        campaignArn = rerank_campaign_arn
    )
    status = version_response['campaign']['status']

    if status == 'ACTIVE':
        print('Build succeeded for {}'.format(rerank_campaign_arn))
    elif status == "CREATE FAILED":
        print('Build failed for {}'.format(rerank_campaign_arn))
        in_progress_campaigns.remove(rerank_campaign_arn)
    
    if status == 'ACTIVE' or status == 'CREATE FAILED':
        break
    else:
        print('The campaign build is still in progress')
        
    time.sleep(60)

The campaign build is still in progress
The campaign build is still in progress
The campaign build is still in progress
The campaign build is still in progress
The campaign build is still in progress
The campaign build is still in progress
The campaign build is still in progress
The campaign build is still in progress
Build succeeded for arn:aws:personalize:us-east-1:061703360474:campaign/personalize-poc-rerank


## Create Filters <a class="anchor" id="interact"></a>
[Back to top](#top)

Amazon Personalize supports the ability to create [filters](https://docs.aws.amazon.com/personalize/latest/dg/filter.html) that can be used to exclude items from being recommended that meet a filter expression. 

Now that all campaigns are deployed and active amd the recommenders have been trained we can create filters. Filters can be created for fields of both Items and Events. Filters can also be created dynamically for `IN` and `=` operations. For range queries, you should use static filters. 
Range queries use the following operations: `NOT IN`, `<`, `>`, `<=`, and `>=`.

A few common use cases for static filters in Video On Demand are:

Categorical filters based on Item Metadata (that are range based) - Often your item metadata will have information about the title such as year, user rating, available date. Filtering on these can provide recommendations within that data, such as movies that are available after a specific date, movies rated over 3 stars, movies from the 1990s etc.

User Demographic ranges - you may want to recommend content to specific age demographics, for this you can create a filter that is specific to a age range like over 18, over 18 AND under 30, etc).

Lets look at the item metadata and user interactions, so we can get an idea what type of filters we can create.

In [24]:
# Create a dataframe for the items by reading in the correct source CSV
items_meta_df = pd.read_csv(data_dir + '/item-meta.csv', sep=',', index_col=0)

# Render some sample data
items_meta_df.head(10)

,GENRES,YEAR,CREATION_TIMESTAMP
ITEM_ID,,,
1,Adventure|Animation|Children|Comedy|Fantasy,1995,1640995200
2,Adventure|Children|Fantasy,1995,1640995200
3,Comedy|Romance,1995,1640995200
4,Comedy|Drama|Romance,1995,1640995200
5,Comedy,1995,1640995200
6,Action|Crime|Thriller,1995,1640995200
7,Comedy|Romance,1995,1640995200
8,Adventure|Children,1995,1640995200
9,Action,1995,1640995200


Since there are a lot of genres to filter on, we will create a dynamic filter using the dynamic variable $GENRE, this will allow us to pass in the variable at runtime rather than create a static filter for each genre.

In [25]:
create_genre_filter_response = personalize.create_filter(name='Genre',
    datasetGroupArn = dataset_group_arn,
    filterExpression = 'INCLUDE ItemID WHERE Items.GENRES IN ($GENRE)'
    )

In [26]:
genre_filter_arn = create_genre_filter_response['filterArn']

Since we have added the year to our item metadata, lets create a decade filter to recommend only movies released in a given decade. A soft limit of Personalize at this time is 10 total filters, so we will create 7 decade filters for this workshop, leaving room for additional static and dynamic filters.

Create a list for the metadata decade filters and then create the actual filters with the cells below. Note this will take a few minutes to complete.

In [27]:
decades_to_filter = [1950,1960,1970,1980,1990,2000,2010]

In [28]:
# Create a list for the filters:
meta_filter_decade_arns = []

In [29]:
# Iterate through Genres
for decade in decades_to_filter:
    # Start by creating a filter
    current_decade = str(decade)
    next_decade = str(decade + 10)
    try:
        createfilter_response = personalize.create_filter(
            name = current_decade + "s",
            datasetGroupArn = dataset_group_arn,
            filterExpression = 'INCLUDE ItemID WHERE Items.YEAR >= '+ current_decade +' AND Items.YEAR < '+ next_decade +''
    )
        # Add the ARN to the list
        meta_filter_decade_arns.append(createfilter_response['filterArn'])
        print("Creating: " + createfilter_response['filterArn'])
    
    # If this fails, wait a bit
    except ClientError as error:
        # Here we only care about raising if it isnt the throttling issue
        if error.response['Error']['Code'] != 'LimitExceededException':
            print(error)
        else:    
            time.sleep(120)
            createfilter_response = personalize.create_filter(
                name = current_decade + "s",
                datasetGroupArn = dataset_group_arn,
                filterExpression = 'INCLUDE ItemID WHERE Items.YEAR >= '+ current_decade +' AND Items.YEAR < '+ next_decade +''
    )
            # Add the ARN to the list
            meta_filter_decade_arns.append(createfilter_response['filterArn'])
            print("Creating: " + createfilter_response['filterArn'])

Creating: arn:aws:personalize:us-east-1:061703360474:filter/1950s
Creating: arn:aws:personalize:us-east-1:061703360474:filter/1960s
Creating: arn:aws:personalize:us-east-1:061703360474:filter/1970s
Creating: arn:aws:personalize:us-east-1:061703360474:filter/1980s
Creating: arn:aws:personalize:us-east-1:061703360474:filter/1990s
Creating: arn:aws:personalize:us-east-1:061703360474:filter/2000s
Creating: arn:aws:personalize:us-east-1:061703360474:filter/2010s


Lets also create 2 event filters for watched and unwatched content. The [Top picks for you](https://docs.aws.amazon.com/personalize/latest/dg/VIDEO_ON_DEMAND-use-cases.html#top-picks-use-case) and [More like X](https://docs.aws.amazon.com/personalize/latest/dg/VIDEO_ON_DEMAND-use-cases.html#more-like-y-use-case) already have a filter on implemented to filter out watched events.

In [34]:
print(data_dir)

poc_data


In [35]:
# Create a dataframe for the interactions by reading in the correct source CSV
interactions_df = pd.read_csv(data_dir + '/interactions.csv', sep=',', index_col=0)

# Render some sample data
interactions_df.head(10)

,ITEM_ID,TIMESTAMP,EVENT_TYPE
USER_ID,,,
2262,1079,789652009,Click
2262,47,789652009,Click
2262,21,789652009,Click
2262,47,789652009,Watch
102689,47,822873600,Click
109832,2,822873600,Click
102689,50,822873600,Watch
109832,60,822873600,Watch
109832,50,822873600,Watch


In [36]:
createwatchedfilter_response = personalize.create_filter(name='watched',
    datasetGroupArn = dataset_group_arn,
    filterExpression = 'INCLUDE ItemID WHERE Interactions.event_type IN ("Watch")'
    )

createunwatchedfilter_response = personalize.create_filter(name='unwatched',
    datasetGroupArn = dataset_group_arn,
    filterExpression = 'EXCLUDE ItemID WHERE Interactions.event_type IN ("Watch")'
    )

ResourceAlreadyExistsException: An error occurred (ResourceAlreadyExistsException) when calling the CreateFilter operation: Another filter with name watched already exists.

Before we move on we want to add those filters to a list as well so they can be used later.

In [37]:
interaction_filter_arns = [createwatchedfilter_response['filterArn'], createunwatchedfilter_response['filterArn']]

## Storing useful variables <a class="anchor" id="vars"></a>
[Back to top](#top)

Before exiting this notebook, run the following cells to save the version ARNs for use in the next notebook.

In [ ]:
itemaffinity_attribute_solution_version_arn

In [57]:
%store rerank_solution_version_arn
%store recommender_top_picks_arn
%store recommender_more_like_x_arn
%store rerank_solution_arn
%store rerank_campaign_arn
%store meta_filter_decade_arns
%store genre_filter_arn
%store interaction_filter_arns
%store itemaffinity_attribute_solution_arn
%store itemaffinity_attribute_solution_version_arn
%store itemaffinity_solution_arn
%store itemaffinity_solution_version_arn

Stored 'rerank_solution_version_arn' (str)
Stored 'recommender_top_picks_arn' (str)
Stored 'recommender_more_like_x_arn' (str)
Stored 'rerank_solution_arn' (str)
Stored 'rerank_campaign_arn' (str)
Stored 'meta_filter_decade_arns' (list)
Stored 'genre_filter_arn' (str)
Stored 'interaction_filter_arns' (list)
Stored 'itemaffinity_attribute_solution_arn' (str)
Stored 'itemaffinity_attribute_solution_version_arn' (str)
Stored 'itemaffinity_solution_arn' (str)
Stored 'itemaffinity_solution_version_arn' (str)


You're all set to move on to the last exploratory notebook: `03_Inference_Layer.ipynb`. Open it from the browser and you can start interacting with the Recommenders and Campaign and getting recommendations!